# GenealogyBot

Le but de ce bot est d'extraire les données généalogiques provenant l'extraction du recensement de 1832 dans la ville de Lausanne et de créer des pages correspondantes sur wikipast en respectant la bonne syntaxe.

Le document original est accessible sur le [site des archives de Lausanne](https://mipade.mediancer.com/avl/avl_viewer.html?mag=14918). 


## 1 Connexion au bot sur wikipast

Connexion sur wikipast avec l'identifant et le mot de passe créé pour le bot GenealogyBot.

In [ ]:
from pywikiapi import Site

user = 'JDezalos@GenealogyBot'
password = 'vu12lg9or06k5pnv1lpur3ou45sd7mat' #critical security breach

site = Site('http://wikipast.epfl.ch/wikipast/api.php') # Définition de l'adresse de l'API
site.no_ssl = True  # Désactivation du https, car pas activé sur wikipast
site.login(user, password)  # Login du bot sur wikipast

## 2 Extraction des données

In [ ]:
import pandas as pd

from IPython.display import display
from urllib.parse import quote

#Voir dataset_parser.py

## 3 Vérification d'existance

In [ ]:
all_parent = pd.read_csv("../data/filtered.csv")

In [ ]:
pages_names = all_parent.apply(lambda x: x.chef_prenom.title() +" "+ x.chef_nom.title() + " né en " + str(x.chef_annee_naissance), axis=1)
results_pages = []
for page in pages_names:
    # Si on veut plus d'info sur les pages qui existent, ajouter à prop
    # Voir http://wikipast.epfl.ch/wikipast/api.php?action=help&modules=query
    for res in site.query_pages(titles=page, prop=["contributors"]):  
        results_pages.append(res)
results_pages

In [ ]:
def create_parent(content):
    prenom = content['chef_prenom'].title()
    nom = content['chef_nom'].title()
    date_naissance = content['chef_annee_naissance']
    titre = '%s %s %s %s'%(prenom, nom, " né en ", date_naissance)

    new_page_father(titre, prenom, nom, date_naissance, source)

In [ ]:
all_child = pd.read_csv("../data/chef_enfant.csv")

In [ ]:
child_pages_names = all_child.apply(lambda x: x.enfants_dans_la_commune_prenom.title() +" "+ x.chef_nom.title() + " né en " + str(x.annee_enfant), axis=1)

child_results_pages = []
for page in child_pages_names:
    # Si on veut plus d'info sur les pages qui existent, ajouter à prop
    # Voir http://wikipast.epfl.ch/wikipast/api.php?action=help&modules=query
    for res in site.query_pages(titles=page, prop=["contributors"]):  
        child_results_pages.append(res)
child_results_pages

In [ ]:
def check_if_parent_is_missing(content):
    prenom_enfant = content['enfants_dans_la_commune_prenom'].title()
    date_naissance_enfant = content['annee_enfant']
    nom_pere = content['chef_nom'].title()
    prenom_pere = content['chef_prenom'].title()
    nom_mere = content['epouse_nom'].title()
    date_naissance_pere = content['chef_annee_naissance']
    titre = '%s %s %s %s'%(prenom_pere, nom_pere, " né en ", date_naissance_pere)

    entry_birth_children(titre, prenom_enfant, date_naissance_enfant, nom_pere, prenom_pere, nom_mere, date_naissance_pere, source)
        
def create_child(content):
    prenom_enfant = content['enfants_dans_la_commune_prenom'].title()
    date_naissance_enfant = content['annee_enfant']
    nom_pere = content['chef_nom'].title()
    prenom_pere = content['chef_prenom'].title()
    nom_mere = content['epouse_nom'].title()
    date_naissance_pere = content['chef_annee_naissance']
    titre = '%s %s %s %s'%(prenom_enfant, nom_pere, " né en ", date_naissance_enfant)
    
    new_page_children(titre, prenom_enfant, date_naissance_enfant, nom_pere, prenom_pere, nom_mere, date_naissance_pere, source)
    check_if_parent_is_missing(content)
    

In [ ]:
def main():
    missing = [page["title"] for page in results_pages if page.get("missing", False) is True]
    existing_pages = [page for page in results_pages if page.get("missing", False) is False] # existing == not missing donc ça devrait pas être nécessaire
    missing.extend([page["title"] for page in existing_pages if len(page.get("contributors")) == 1 and page.get("contributors")[0].get("name", "") == "JDezalos"])

    all_parent["missing"] = all_parent.apply(lambda x: create_parent(x), axis=1)

    
    
    child_missing = [page["title"] for page in child_results_pages if page.get("missing", False) is True]
    child_existing_pages = [page for page in child_results_pages if page.get("missing", False) is False] # existing == not missing donc ça devrait pas être nécessaire
    child_missing.extend([page["title"] for page in child_existing_pages if len(page.get("contributors")) == 1 and page.get("contributors")[0].get("name", "") == "JDezalos"])

    all_child["missing"] = all_child.apply(lambda x: create_child(x), axis=1)

### 3.1 Création de pages

#### 3.1.1 Création d'une nouvelle page avec l'entrée correspondant à la naissance de la personne

Définition des fonctions "new_page_father" et "new_page_children" permettant de créer respectivement une page pour un chef de famille ou une page pour un enfant avec une entrée pour leur naissance en utilisant la syntaxe correcte :

- pour les chefs de famille : `[[Date]]/ -. [[Naissance]] de [[Prénom Nom]]. [Source]`

- pour les enfants : `[[Date]]/ -. [[Naissance]] de [[Prénom Nom]], enfant de [[Prénom Nom]] et de son épouse née Nom. [Source]`

In [ ]:
def new_page_father(titre, prenom, nom, date_naissance, source):
    text = '\n* [[%s]] / -. [[Naissance]] de [[%s %s né en %s| %s %s]]. [%s]\n'%(date_naissance, prenom, nom, date_naissance, prenom, nom, source)
    site('edit', title=titre, text=text, token=site.token())
    
def new_page_children(titre, prenom_enfant, date_naissance_enfant, nom_pere, prenom_pere, nom_mere, date_naissance_pere, source):
    if nom_mere == "·":
        text = '\n* [[%s]] / -. [[Naissance]] de [[%s %s né en %s| %s %s]], enfant de [[%s %s né en %s| %s %s]]. [%s]\n'%(date_naissance_enfant, prenom_enfant, nom_pere, date_naissance_enfant, prenom_enfant, nom_pere, prenom_pere, nom_pere, date_naissance_pere, prenom_pere, nom_pere, source)
    else:
        text = '\n* [[%s]] / -. [[Naissance]] de [[%s %s né en %s| %s %s]], enfant de [[%s %s né en %s| %s %s]] et de son épouse née %s. [%s]\n'%(date_naissance_enfant, prenom_enfant, nom_pere, date_naissance_enfant, prenom_enfant, nom_pere, prenom_pere, nom_pere, date_naissance_pere, prenom_pere, nom_pere, nom_mere, source)
    site('edit', title=titre, text=text, token=site.token())

Définition de la source des données utilisées (qui est la même pour toutes les entrées).

In [ ]:
source = 'https://mipade.mediancer.com/avl/avl_viewer.html?mag=14918'

#### 3.1.2 Création d'entrées correspondant à la naissance d'un enfant sur la page du chef de famille

Création de la fonction "entry_birth_children" permettant de rajouter à la page du chef de famille une entrée correpondant à la naissance de chacun de ses enfants. La syntaxe utilisée pour ces entrées est exactement la même que celle utilisée lors de la création de la page pour les enfants :

`[[Date]]/ -. [[Naissance]] de [[Prénom Nom]], enfant de [[Prénom Nom]] et de son épouse née Nom. [Source]`

In [ ]:
def entry_birth_children(titre, prenom_enfant, date_naissance_enfant, nom_pere, prenom_pere, nom_mere, date_naissance_pere, source):
    text = '\n* [[%s]] / -. [[Naissance]] de [[%s %s né en %s| %s %s]], enfant de [[%s %s né en %s| %s %s]] et de son épouse née %s. [%s]\n'%(date_naissance_enfant, prenom_enfant, nom_pere, date_naissance_enfant, prenom_enfant, nom_pere, prenom_pere, nom_pere, date_naissance_pere, prenom_pere, nom_pere, nom_mere, source)

    site('edit', title=titre, appendtext=text, token=site.token())

In [ ]:
#Execution du programme
main()